In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

2021-11-14 19:54:24.626240: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


2021-11-14 19:54:32.206033: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-14 19:54:32.230013: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-14 19:54:32.522945: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 19:54:32.523766: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.1

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2021-11-14 19:54:39.462608: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-14 19:54:39.462703: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 19:54:39.463124: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2021-11-14 19:54:39.463168: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 19:54:39.463504: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00

In [3]:
import numpy as np
import pandas as pd

In [4]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [5]:
# 顯示所有列
pd.set_option('display.max_columns', None)

In [6]:
# 讀前處理過後的資料
# train = pd.read_csv('Processed_tbrain_training.csv')

# 讀合併完消費次數的看看會不會比較快
train = pd.read_csv('Processed_tbrain_training.csv')
print(train.shape)
print(train.info())
train.head(5)

(32970085, 53)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32970085 entries, 0 to 32970084
Data columns (total 53 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  int64  
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_cnt            int64  
 14  card_2_txn_cnt            int64  
 15  card_3_txn_cnt            int64  
 16  card_4_txn_cnt            int64  
 17  card_5_txn_cnt            int64  
 18  card_6_txn_cnt            int64  
 19  card_7_txn_cnt            int64  
 20  card_8_

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10321418,45,3,5.830816e-112,3,0,0,0,1.00,0.00,0.0,0.0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,99.0,30.0,1.394080e-213,1.0,4.0,1
1,1,10414574,15,2,1.596363e-111,2,0,0,0,1.00,0.00,0.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0,2.0,30.0,1.944332e-213,1.0,3.0,1
2,1,10134567,48,2,3.541844e-111,0,2,0,0,0.00,1.00,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,3.0,30.0,1.648096e-213,0.0,4.0,1
3,1,10001003,48,9,2.671485e-111,7,2,0,0,0.75,0.25,0.0,0.0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,11.0,1.0,2.0,30.0,8.190756e-214,0.0,3.0,1
4,1,10267183,2,1,3.266644e-111,1,0,0,0,1.00,0.00,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,8.190756e-214,0.0,6.0,0


In [7]:
# 先取出每個顧客不重複最後一個月的值之後預測用
pre_train = train.drop_duplicates(subset=['chid'], keep='last')
pre_train

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
14519147,13,10111801,37,1,3.578584e-112,0,0,1,0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,1.980761e-213,1.0,2.0,0
14519440,13,10013382,26,0,2.872190e-111,0,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,2.0,1.0,1.0,30.0,1.648096e-213,1.0,6.0,1
14519711,13,10464422,2,1,4.450214e-112,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,15.0,1.0,99.0,30.0,2.215702e-213,0.0,5.0,1
14520768,13,10203945,37,1,1.248347e-111,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,8.0,30.0,1.730576e-213,0.0,6.0,1
14522515,13,10276085,37,1,9.176177e-112,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,5.0,1.0,99.0,30.0,1.258009e-213,0.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32970073,24,10430654,6,1,6.649981e-112,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,2.0,1.0,1.0,30.0,1.017454e-213,0.0,6.0,1
32970075,24,10152908,48,1,8.665268e-112,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,9.0,1.0,2.0,30.0,8.190756e-214,0.0,2.0,1
32970077,24,10057828,37,2,2.036889e-111,0,2,0,0,0.0,1.0,0.0,0.0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,9.138293e-214,0.0,6.0,0
32970080,24,10297394,19,1,2.098797e-111,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,15.0,1.0,2.0,30.0,8.190756e-214,0.0,2.0,1


In [8]:
# 去除shop_tag
pre_train.drop(columns = ['shop_tag'], axis=1, inplace=True)

/mnt/md0/home/hongjun/anaconda3/envs/python37/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
len(pre_train)

500000

In [10]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

(500000, 1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   chid    500000 non-null  int64
dtypes: int64(1)
memory usage: 3.8 MB
None


,chid
0,10128239
1,10077943
2,10277876
3,10364842
4,10392717


In [11]:
# 按照要提交的檔案排序
pre_train = pre_train.set_index('chid', drop= False)
pre_train = pre_train.loc[predict.chid]
pre_train.head(5)

,dt,chid,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10128239,24,10128239,2,1.580629e-111,2,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
10077943,24,10077943,2,1.019929e-111,0,0,0,2,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,12.0,1.0,2.0,30.0,7.594606e-214,0.0,5.0,1
10277876,24,10277876,1,3.600542e-112,1,0,0,0,1.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,6.0,1.0,1.0,30.0,7.077406e-214,1.0,3.0,1
10364842,24,10364842,1,3.600542e-112,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,7.0,1.0,99.0,30.0,1.048096e-213,1.0,4.0,1
10392717,24,10392717,1,1.158421e-111,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,5.0,1.0,2.0,30.0,3.248696e-214,1.0,2.0,1


In [12]:
# 拆分測試訓練集
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [13]:
target_column_train = ['shop_tag']
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

(26376068, 52)
(26376068, 1)


In [14]:
target_column_test = ['shop_tag'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

(6594017, 52)
(6594017, 1)


In [15]:
import gc
del train
del test
gc.collect()

69

In [16]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train.shape[1])

(26376068, 52)
(26376068, 1)
(6594017, 52)
(6594017, 1)
52


In [17]:
X_train1 = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [18]:
X_train1.shape

(26376068, 52, 1)

In [19]:
y_train1 = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

In [20]:
y_train1.shape

(26376068, 1, 1)

In [21]:
X_test1 = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test1.shape

(6594017, 52, 1)

In [22]:
y_test1 = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))
y_test1.shape

(6594017, 1, 1)

In [23]:
pre_train = pre_train.values
pre_train = np.reshape(pre_train, (pre_train.shape[0], pre_train.shape[1], 1))
pre_train.shape

(500000, 52, 1)

In [24]:
X_train1.shape[1]

52

In [25]:
# X_train[..., None].shape

In [26]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(49, activation='softmax'))
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

# model = Sequential()
# model.add(LSTM(100, input_shape=(X_train1.shape[1],1 ), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(200))
# model.add(Dropout(0.2))
# model.add(Dense(49, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

2021-11-14 19:59:46.333943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 19:59:46.388415: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2021-11-14 19:59:46.388896: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-14 19:59:46.389924: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 52, 50)            10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 49)                2499      
Total params: 33,099
Trainable params: 33,099
Non-trainable params: 0
_________________________________________________________________


In [27]:
X_train.shape

(26376068, 52)

In [28]:
y_train.shape

(26376068, 1)

In [29]:
y_train[0:300].shape

(300, 1)

In [30]:
np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))[0:300].shape

(300, 1, 52)

In [31]:
np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1]))[0:300].shape

(300, 1, 1)

In [32]:
# tf.config.experimental.list_physical_devices('GPU')

In [33]:
# # # from tensorflow.compat.v1 import ConfigProto
# # # from tensorflow.compat.v1 import InteractiveSession
# # # config = ConfigProto()
# # # config.gpu_options.allow_growth = True
# # # session = InteractiveSession(config=config)


# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)
    

In [34]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [ ]:
# import tensorflow as tf 
model.fit(X_train1, y_train1, epochs=1, batch_size=1, verbose=1)
# model.fit(X_train1[0:300],y_train1[0:300], validation_data = (X_test1, y_test1),
#           batch_size=1, epochs=1)

2021-11-14 19:59:51.560638: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 5486222144 exceeds 10% of free system memory.
2021-11-14 19:59:57.154059: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-14 19:59:57.235076: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3199980000 Hz
2021-11-14 20:00:00.115502: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-11-14 20:00:02.280732: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


 2915503/26376068 [==>...........................] - ETA: 17:22:15 - loss: 2.7426 - accuracy: 0.2223

In [ ]:
# 模型預測
predictions = model.predict(pre_train)

In [ ]:
len(predictions)

In [ ]:
len(predictions[0])

In [ ]:
# 按降序獲取前 n 個結果
best_n = np.argsort(-predictions, axis=1)[:, :49]
best_n

In [ ]:
# 建立best_predictions_final來儲存前三高的預測值
best_predictions_final = np.zeros((len(best_n), 3), dtype=int)

In [ ]:
# 過濾16個前三高的標籤出來
for a in range(len(best_n)):
    sum = 0
    for i in range(49): 
        if (best_n[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,39,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_n[a, i])
            sum += 1 
        if sum == 3: 
            break

In [ ]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [ ]:
filename = 'submission-不砍欄位.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)

In [ ]:
# 保存模型
model.save('LSTM_model_all-不砍欄位-1.h5') 